Cite as: Boeing, G. 2021. “Street Network Models and Indicators for Every Urban Area in the World.” Geographical Analysis, published online before print. doi:10.1111/gean.12281

In [1]:
import numpy as np
import osmnx as ox
import folium
from IPython.display import IFrame

%matplotlib inline
np.random.seed(0)
ox.__version__

'1.2.1'

Carga los datos de la ciudad

In [2]:
place = "Bogota, Colombia"
G = ox.graph_from_place(place, network_type="drive")
Gp = ox.project_graph(G)

## 2. Basic routing by distance

Pick two nodes. Then find the shortest path between origin and destination, using weight='length' to find the shortest path by minimizing distance traveled (otherwise it treats each edge as weight=1).

In [3]:
# find the shortest path (by distance) between these nodes then plot it
orig = list(G)[0]
dest = list(G)[190]

routes = ox.k_shortest_paths(G, orig, dest, k=1, weight="length") # k is the number of shortest paths
#fig, ax = ox.plot_graph_routes(G, list(routes), route_colors="y", route_linewidth=4, node_size=0)

## 3. Imputing travel speeds and times

The `add_edge_speeds` function add edge speeds (km per hour) to graph as new `speed_kph` edge attributes. Imputes free-flow travel speeds for all edges based on mean `maxspeed` value of edges, per highway type. This mean-imputation can obviously be imprecise, and the caller can override it by passing in `hwy_speeds` and/or `fallback` arguments that correspond to local speed limit standards. See docstring for details.

In [4]:
# impute speed on all edges missing data
G = ox.add_edge_speeds(G)

# calculate travel time (seconds) for all edges
G = ox.add_edge_travel_times(G)

In [5]:
# see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

,length,speed_kph,travel_time
highway,,,
"['primary', 'secondary']",230.6,38.7,21.5
"['primary_link', 'primary']",53.8,34.0,5.7
"['secondary', 'tertiary']",570.1,31.7,64.8
"['secondary', 'tertiary_link']",1576.9,31.7,179.1
"['tertiary', 'residential']",219.2,30.2,26.2
"['unclassified', 'residential']",209.1,30.0,25.1
living_street,16.3,30.0,2.0
primary,128.6,38.7,12.3
primary_link,94.7,34.0,10.1


In [6]:
# same thing again, but this time pass in a few default speed values (km/hour)
# to fill in edges with missing `maxspeed` from OSM
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

In [7]:
# calculate two routes by minimizing travel distance vs travel time
orig = list(G)[1]
dest = list(G)[120]
route1 = ox.shortest_path(G, orig, dest, weight="length")
route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

In [8]:
# compare the two routes
route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "length")))
route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time")))
route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "travel_time")))
print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")

Route 1 is 9654 meters and takes 665 seconds.
Route 2 is 9660 meters and takes 648 seconds.


Plot a city's street network as an interactive web map

In [9]:
# m1 = ox.plot_graph_folium(G, popup_attribute="name", weight=2, color="#8b0000")
# m1

KeyboardInterrupt: 

Plot a route as an interactive web map

In [27]:
lats = [G.nodes[n]['y'] for n in route1]
longs = [G.nodes[n]['x'] for n in route1]

coords = list(zip(lats, longs))

In [30]:
coords

[(4.6644634, -74.0572856),
 (4.6636778, -74.0577435),
 (4.6635168, -74.0578584),
 (4.6638293, -74.0585753),
 (4.6644415, -74.0596512),
 (4.6648681, -74.060236),
 (4.6654093, -74.0601446),
 (4.6666031, -74.0599363),
 (4.6679328, -74.0597361),
 (4.6680109, -74.0600875),
 (4.6682542, -74.0600374),
 (4.6694987, -74.0599401),
 (4.6760076, -74.0588698),
 (4.6805331, -74.0580911),
 (4.6841697, -74.0575043),
 (4.6846728, -74.0574182),
 (4.6887836, -74.0567849),
 (4.6922174, -74.0561946),
 (4.6976512, -74.0552587),
 (4.6977452, -74.0552409),
 (4.7012473, -74.0546667),
 (4.7044185, -74.0541404),
 (4.7114748, -74.0529708),
 (4.7132485, -74.0526607),
 (4.7222555, -74.0511208),
 (4.7280512, -74.0502054),
 (4.7292112, -74.0497796),
 (4.7298494, -74.0496711),
 (4.7303648, -74.049584),
 (4.7306182, -74.0495382),
 (4.7308846, -74.0494886),
 (4.7319896, -74.0493044),
 (4.7326411, -74.0491908),
 (4.7346662, -74.0488527),
 (4.7364014, -74.0485756),
 (4.7366805, -74.048524),
 (4.7373859, -74.0483918),
 (4.

In [28]:
mc = folium.Map(coords[0], zoom_start=12)

In [33]:
import plotly
import pandas as pd

In [36]:
df = pd.DataFrame(coords, columns=['lat', 'lon'])

In [41]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="lat", lon="lon",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [29]:
folium.PolyLine(coords, color="red", weight=2).add_to(mc)
mc

In [17]:
G.nodes[253761879]

{'y': 4.6714483, 'x': -74.0579512, 'street_count': 3}

In [10]:
m2 = ox.plot_route_folium(G, route1, weight=1)


In [11]:
folium.Marker([4.6636778, -74.0577435]).add_to(m2)
folium.Marker([4.5025163,-74.0798766]).add_to(m2)
folium.Marker([4.5019653, -74.0806055]).add_to(m2)


In [14]:
route1

AttributeError: 'list' object has no attribute 'to_crs'

In [12]:
m2

Plot a route on top of the complete street network map

In [12]:
# m3 = ox.plot_route_folium(G, route1,route_map=m1, popup_attribute="length", weight=7)
# m3

In [13]:
print(list(G)[1]) #muestra el primer nodo de la ruta G

253761882


In [14]:
print(G[253761882]) #muestra la info para el nodo X

{253762016: {0: {'osmid': 89615978, 'oneway': True, 'lanes': '2', 'name': 'Carrera 14', 'highway': 'tertiary', 'maxspeed': '30', 'reversed': False, 'length': 101.026, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001F86C88CEB0>, 'speed_kph': 30.0, 'travel_time': 12.1}}}


Mostrar la longitud y latitud para un nodo

In [15]:
print(G._node[253761882],G._node[253762016])

{'y': 4.6644634, 'x': -74.0572856, 'street_count': 3} {'y': 4.6636778, 'x': -74.0577435, 'street_count': 3}


In [16]:
print(route1) #muestra los nodos de la ruta

[253761882, 253762016, 7989948688, 253762062, 264441789, 851224205, 312645998, 5709996738, 260623191, 3879238831, 5113324383, 5709996784, 5712218348, 5712241742, 5712241757, 5712269454, 3582151544, 368396518, 5712302400, 5712302345, 5712302420, 368396876, 1884674800, 369183559, 746004274, 392198530, 5712367235, 312479385, 312479388, 9396229672, 312479403, 312479394, 368397767, 310343239, 551108947, 310711504, 310711644, 310343240, 763664225, 506225166, 368398165, 551108950, 329167185, 2331687713, 310707735, 310707736, 310707753, 310707787, 9359376547, 9359376548, 312410688, 312410487, 259709273]


In [20]:
ox.distance.nearest_edges(G, 4.6636778, -74.0577435, interpolate=None, return_dist=False)



c:\Users\alejo\AppData\Local\Programs\Python\Python310\lib\site-packages\osmnx\distance.py:304: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dists = geoms.iloc[list(rtree.nearest(xy, num_results=10))].distance(Point(xy))


(1326492088, 1326483089, 0)

In [21]:
ox.distance.nearest_edges(G, 4.6644634, -74.0572856, interpolate=None, return_dist=False)

c:\Users\alejo\AppData\Local\Programs\Python\Python310\lib\site-packages\osmnx\distance.py:304: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dists = geoms.iloc[list(rtree.nearest(xy, num_results=10))].distance(Point(xy))


(1326492088, 1326483089, 0)

In [29]:
print(G._node[1326492088], G._node[1326483089])

{'y': 4.5025163, 'x': -74.0798766, 'street_count': 3} {'y': 4.5019653, 'x': -74.0806055, 'street_count': 3}


In [38]:
G.in_edges(route1)

InMultiEdgeDataView([(253847647, 253761882), (267890885, 253761882), (253761882, 253762016), (267890883, 253762016), (253762016, 7989948688), (851224193, 253762062), (7989948688, 253762062), (253762062, 264441789), (312645997, 264441789), (260627307, 851224205), (264441789, 851224205), (312645962, 312645998), (851224205, 312645998), (312645325, 5709996738), (312645998, 5709996738), (312645765, 260623191), (5709996738, 260623191), (260623191, 3879238831), (3879238830, 3879238831), (3879238831, 5113324383), (5113324383, 5709996784), (5719108233, 5709996784), (5709996784, 5712218348), (5712218345, 5712218348), (5712218348, 5712241742), (268152954, 5712241757), (5712241742, 5712241757), (5712241757, 5712269454), (5220581404, 3582151544), (5712269454, 3582151544), (368396516, 368396518), (3582151544, 368396518), (368396518, 5712302400), (5712302400, 5712302345), (368396838, 5712302420), (5712302345, 5712302420), (368396874, 368396876), (5712302420, 368396876), (368396876, 1884674800), (1884